In [ ]:
#importing google drive to load dataset
from google.colab import drive

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing required libraries.
#Note : only required libraries like nltk, pandas for simplification are used. No sci-kit libraries like naive bayes are used.

import pandas as pd
from collections import Counter
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download('punkt')
from nltk.corpus import stopwords
from collections import defaultdict
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#loading json file in a variable df

df = pd.read_json('/content/drive/MyDrive/dataset.json',lines=True)
df

,short_description,headline,date,link,authors,category
0,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,2018-05-26,https://www.huffingtonpost.com/entry/texas-ama...,Melissa Jeltsen,CRIME
1,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,2018-05-26,https://www.huffingtonpost.com/entry/will-smit...,Andy McDonald,ENTERTAINMENT
2,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 57,2018-05-26,https://www.huffingtonpost.com/entry/hugh-gran...,Ron Dicker,ENTERTAINMENT
3,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,2018-05-26,https://www.huffingtonpost.com/entry/jim-carre...,Ron Dicker,ENTERTAINMENT
4,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,2018-05-26,https://www.huffingtonpost.com/entry/julianna-...,Ron Dicker,ENTERTAINMENT
...,...,...,...,...,...,...
124984,,Why I Thought I'd Never Live To See 33,2014-04-18,https://www.huffingtonpost.com/entry/a-black-g...,,WOMEN
124985,I often hear people describe cooking for one a...,Five Tips For Cooking For One,2014-04-18,https://www.huffingtonpost.com/entry/five-tips...,"Food Riot, ContributorPlay with your food.",TASTE
124986,,Police Want your Home Video Surveillance Footage,2014-04-18,https://www.huffingtonpost.com/entry/police-wa...,"Robert Siciliano, ContributorPersonal Security...",CRIME
124987,,"Ouch, Natalie Morales",2014-04-18,https://www.huffingtonpost.com/entry/natalie-m...,Katherine Fung,MEDIA


In [ ]:
#extracting only the required the given categories

filtered_df = df[df['category'].isin(['COMEDY','BUSINESS', 'SPORTS', 'CRIME','RELIGION','HEALTHY LIVING', 'POLITICS'])]
filtered_df_shuffle = filtered_df.sample(frac=1,random_state=42)

total_rows = len(filtered_df_shuffle)
train_rows = int(0.85*total_rows)
test_rows = total_rows-train_rows
#dividing the data into 85% training and 15% testing

train_df = filtered_df_shuffle.iloc[:train_rows, :]
test_df = filtered_df_shuffle.iloc[train_rows:, :]

In [ ]:
#finding total no of unique categories. (for training)

uniq_categ = train_df['category'].value_counts()
total =0
for i in range(0,len(uniq_categ)):
  total+=uniq_categ[i]

uniq_categ


POLITICS          27849
HEALTHY LIVING     5663
BUSINESS           3625
SPORTS             3543
COMEDY             3333
CRIME              2502
RELIGION           2167
Name: category, dtype: int64

In [ ]:
#finding total no of uniques categories. (for testing)

uniq_cat = test_df['category'].value_counts()
total_test =0
for i in range(0,len(uniq_cat)):
  total_test+=uniq_cat[i]

total_test

8592

In [ ]:
#finding total no of words in each category and calculating prior probabilites of each category. (for training)

prior = {}
for category,count in uniq_categ.items():
  prob = count/total
  prior[category] = {'count':count,'prob': prob}

for category in uniq_categ.items():
  print(category[0])

POLITICS
HEALTHY LIVING
BUSINESS
SPORTS
COMEDY
CRIME
RELIGION


In [ ]:
#finding total no of words in each category and calculating prior probabilites of each category. (for testing)

prior_test = {}
for category,count in uniq_cat.items():
  prob = count/total_test
  prior_test[category] = {'count':count,'prob': prob}

for category in uniq_cat.items():
  print(category[0])

POLITICS
HEALTHY LIVING
COMEDY
BUSINESS
SPORTS
CRIME
RELIGION


In [ ]:
# we are calculating count of each word in each category.
# for this we remove all the stopwords, punctuations and join the all the descriptions belonging to a particular cateogry into single string.
# then all we are left with are content words.

all_desc = ' '.join(train_df['short_description'])
all_words = re.findall(r'\b\w+\b',all_desc.lower())
categ_word_count = {}
unique_words_count = {}
filtered= []
import string
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)


#for each word we are calculating its count in a particular category.
# basically that's we are doing in this following lines

for category in train_df['category'].unique():
  category_df = train_df[train_df['category']==category]

  category_words = ' '.join(category_df['short_description'])
  words = word_tokenize(category_words.lower())

  content_words = [word for word in words if word.isalpha() and word not in stop_words]
  unique_words = set(content_words)
  filtered = content_words
  word_freq = FreqDist(content_words)
  categ_word_count[category] = word_freq
  unique_words_count[category] = len(unique_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
categ_word_count

{'POLITICS': FreqDist({'trump': 3312, 'president': 1988, 'said': 1710, 'one': 1231, 'donald': 1168, 'people': 1128, 'would': 1117, 'new': 1078, 'state': 937, 'house': 772, ...}),
 'HEALTHY LIVING': FreqDist({'life': 422, 'one': 376, 'people': 348, 'health': 342, 'time': 339, 'us': 290, 'new': 251, 'may': 241, 'make': 230, 'like': 230, ...}),
 'COMEDY': FreqDist({'like': 162, 'trump': 138, 'one': 109, 'host': 109, 'show': 105, 'president': 100, 'people': 91, 'get': 85, 'late': 78, 'know': 74, ...}),
 'BUSINESS': FreqDist({'business': 278, 'new': 270, 'people': 216, 'one': 211, 'company': 194, 'time': 172, 'many': 156, 'work': 153, 'make': 151, 'world': 133, ...}),
 'SPORTS': FreqDist({'team': 138, 'one': 130, 'said': 118, 'like': 114, 'game': 114, 'first': 107, 'football': 101, 'new': 98, 'nfl': 98, 'sports': 92, ...}),
 'CRIME': FreqDist({'police': 261, 'said': 216, 'people': 105, 'one': 98, 'man': 90, 'say': 88, 'two': 86, 'suspect': 82, 'shot': 79, 'shooting': 73, ...}),
 'RELIGION':

In [ ]:
unique_words_count

{'POLITICS': 21035,
 'HEALTHY LIVING': 10058,
 'COMEDY': 5881,
 'BUSINESS': 9180,
 'SPORTS': 6789,
 'CRIME': 5426,
 'RELIGION': 6443}

In [ ]:
# in this following lines what we are doing is :
# for each row in training set we are converting its description into a single string
# then for each category, we are iterating over this string and calculating the probability of each word
# occuring in that category by the Multinomial formula

correct =0
category_count={}
category_correct={}
print("Category-wise accuracy : ")
for category in uniq_categ.items():
  category_count[category[0]] = 0  # Initialize the count for each category
  category_correct[category[0]] = 0  # Initialize the correct prediction count for each category
for i in range(0,len(test_df)):
  #print(test_df.iloc[i])
  string = test_df.iloc[i]['short_description']
  cat = test_df.iloc[i]['category']
  words1 = re.findall(r'\b\w+\b',string.lower())
  words = word_tokenize(string.lower())
  word = set(words)
  res_prob={}
  for category in uniq_categ.items():
    group_prob=np.log(prior[category[0]]['prob'])
    for w in word:
      word_count = categ_word_count[category[0]][w]
      res = np.log((1+word_count))-np.log((10000*unique_words_count[category[0]]+category[1])) # applying the multinomial formula
      group_prob+=res

    res_prob[category[0]]= group_prob

  #print(res_prob)
  max_categ = max(res_prob,key=res_prob.get) # finding category with max probability.
  #category_count[category[0]]+=1
  # print(max_categ)
  if cat==max_categ:
    category_correct[max_categ]+=1
    correct+=1
  #print("predicted: ",max_categ)
  #print("---------------------------------------")
for category in uniq_categ.items():
  print(category[0]," = ",((category_correct[category[0]])/(prior_test[category[0]]['count']))*100)
print("Overall Accuracy : ",((correct)/len(test_df))*100)








Category-wise accuracy : 
POLITICS  =  59.6319018404908
HEALTHY LIVING  =  42.09505334626576
BUSINESS  =  29.88871224165342
SPORTS  =  41.02564102564102
COMEDY  =  42.31974921630094
CRIME  =  60.61381074168798
RELIGION  =  38.56041131105398
Overall Accuracy :  51.80400372439479
